# Introduktion til topic modelling med latent dirichlet allocation (LDA)

Indtil videre har vi kigget på teknikker, der hjælper os til at få et overblik over ordene, som indgår i data. Disse teknikker egner sig godt til det, som man kalder "nøgleordsanalyse"; altså en analyse af de mest centrale ord/termer i et corpus.

Vi kan dog også inddrage teknikker til at hjælpe os identificere umiddelbare mønstre og temaer i en samling af tekster.

"Latent Dirichlet Allocaltion" (LDA) ([Blei, David M. 2012](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf)): - en af de mere populære "topic models" - er en måde at anvende usuperviseret maskinlæring til at finde umiddelbare temaer i en tekst.

**Fordele**
- Giver overblik over umiddelbare tematiske mønstre i data
- Hjælper til at inddele og kategorisere tekstdata

**Ulemper**
- Fremgangsmåden præget af vilkårlighed (hvor mange topics giver mening at finde?)
- Sårbar over for små ændringer i datahåndtering (hvorvidt man bruger lemmatizer, hvilke stopord man frasorterer)
- Ringe ekstern validitet: Modellen siger noget om den pågældende tekstsamling, men kan i ringe grade anvendes på andre tekstdata


## Hvordan fungerer topic modelling (LDA)?

LDA er en såkaldt "bag-of-words" model. Det vil først og fremmest sige, at den ikke tager højde for ordets sætningskontekst, men blot behandler en tekst som en samling af enkeltord.

![bow](.././img/bow.jpeg)

*[ProgrammerSought.com 2021](https://www.programmersought.com/article/4304366575/)*

### LDA kort fortal

![lda](.././img/lda.png)

- Det antages, at der eksisterer et vis antal "topics", der går igen på tværs af alle dokumenter i et corpus
- Et "topic" er en sandsynlighedsfordeling over en række ord; en samling af ord behæftet med en hvis sandsynlighed
- Hvert dokument antages at være sammensat af de forskellige topics; hvert topic er behæftet med en hvis sandsynlighed for at optræde i dokumentet
- Ordene i et dokument antages at være "trukket" af de forskellige topics

Lad os fx antage, at reddit kommentarer består af tre topics: konspiration, forretning og politik. Antagelsen i LDA er så, at en reddit kommentar, som nedenstående, er dannet ud fra en fordeling af disse topics:

> Der er en ikke helt ude konspirations teori der bygger på at de store firmaer støttede det så de slap for at blive beskattet, ved at distrahere venstrefløjen.

Lad os antage, at en topic model betragter denne kommentar som værende 40% konspiration, 25% forretning og 35% politik. Ordene i kommentaren antages at være taget fra denne fordeling af topics. I hvert topic er visse ord mere sandsynlige end andre ("konspiration" mere sandynsligt i konspirations-topic end forretning-topic, "venstrefløjen" mere sandsynligt i politik-topic end konspirations-topic, osv.).


### LDA som en "generativ probabilitisk model"

LDA er en "generativ probabilitisk model". Det vil sige, at data antages at blive dannet af en generativ proces over en sandsynlighedsfordeling, som er sammensat af *både* observerede og uobserverede variable ([Blei, David M. 2012](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf)).

De observerede variable for LDA er ordene i teksterne, mens de uoverserverede variable er de topics, som dokumenterne antages at være dannet af.

Sandsynlighedsfordelingen over ord og topics i en samling dokumenter noteres som følgende:

$$
p(\beta_{1:K}, \theta_{1:D}, z_{1:D}, w_{1:d})
$$
$$
= \prod_{i=1}^{K}p(\beta_i)\prod_{d=1}^{D}p(\theta_d)\Big(\prod_{n=1}^{N}p(z_{d,n}|\theta_d)p(w_{d,n}|\beta_{1:K},z_{d,n})\Big)
$$

- $\beta_{1:K}$ er topics,  hvor hvert $\beta_{k}$ er en sandsynlighedsfordeling af ord
- $\theta_{d,k}$ er topic-andel for topic $k$ for tekst $d$
- $z_{d,n}$ er tildeling af topic for $n$ ord i tekst $d$
- $w_{d,n}$ er observerede ord $n$ for dokument $d$


Da topic-strukturen er *uobserveret* variabel, kan modellen i sig selv ikke give svar på, hvilke emner, som der findes. Vi kan dog med modellen definere topic-strukturen ved at specificere et antal topics, hvorefter modellen så kan foretage beregningen for ords sandsynligheder i de enkelte topics og sandsynlighederne for topics i dokumenterne.

Der findes dog forskellige teknikker, der forsøger at løse problemet om at finde den optimale eller mest passende topic struktur.

## Brug af topic models i Python

I det følgende ser vi på, hvordan vi kan lave en topic model. Vi bruger pakken [`gensim`](https://radimrehurek.com/gensim/) til formålet.

`gensim` er en meget udbredt pakke til topic modelling. Dog er den ikke udviklet til at være kompatibel med `pandas`, hvorfor vi er nødt til at konvertere data til at være kompatibel med `gensim`.

Den datastruktur, som `gensim` skal bruge, er et "gensim corpus". Et "gensim corpus" er en liste af token-optællinger for hver tekst i et corpus. Sådan et corpus dannes ved først at konvertere hvert token i corpus til et numerisk id (hvert token i corpus får unikt numerisk id). Derefter tælles antallet af tokens per tekst, og samles i tuples bestående af det numeriske id og antal hændelser i teksten.

### Fra tekster til gensim corpus

Til at starte med, skal vi bruge lister af tokens for hver tekst. Disse har vi allerede i vores dataframe (den, som ikke er konverteret til tidy). Vi kan derfor tage disse lister af tokens ud og arbejde med dem for sig.

In [5]:
import pandas as pd
import ast

tweets_df = pd.read_csv('https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/poltweets_sample_tokens.csv')
tweets_df['tokens'] = tweets_df['tokens'].apply(ast.literal_eval)
tweets_df.head()

,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,urls,user_followers_count,party,tokens,dominant_topic_dict
0,2020-10-21 14:48:39+00:00,1318927184111730700,Er på vej i miljøministeriet for at foreslå at...,False,13,47,False,False,False,['dkgreen'],[],4064,Alternativet,"[vej, miljøministerium, biodiversitetsmål, Dan...","{'dominant_topic': 4, 'topic_probability': 0.8..."
1,2019-06-02 20:03:20+00:00,1135275725592891400,@nielscallesoe @helenehagel @alternativet_ Det...,False,0,1,False,False,False,[],[],4064,Alternativet,"[æalternativ, nettofordel, klima, boring, Nord...","{'dominant_topic': 8, 'topic_probability': 0.8..."
2,2016-03-10 09:07:52+00:00,707855478320189400,"Vi står sammen, smiler Løkke på KL-topmøde og ...",False,13,14,False,False,False,"['dkpol', 'KLtop16']",[],4064,Alternativet,"[Løkke, KL-topmøde, milliard, kommune, æaltern...","{'dominant_topic': 0, 'topic_probability': 0.8..."
3,2019-04-07 19:59:03+00:00,1114980930467315700,@AnnaBylov @EU_Spring @rasmusnordqvist 💚,False,0,2,False,False,False,[],[],4064,Alternativet,[eu_spring],"{'dominant_topic': 3, 'topic_probability': 0.5..."
4,2017-05-28 09:59:26+00:00,868768670427828200,Der er ikke noget alternativ til at Alternativ...,False,6,28,False,False,False,['LMÅ17'],"[{'url': 'https://t.co/3MCdZZGKRq', 'expanded_...",4064,Alternativet,"[alternativ, alternativ, klima, æalternativ]","{'dominant_topic': 9, 'topic_probability': 0.4..."


In [6]:
import gensim

# Lagr tweet tokens i liste for sig
tweets_tokens = list(tweets_df['tokens'])

print(tweets_tokens[0])

['vej', 'miljøministerium', 'biodiversitetsmål', 'Danmark', 'natur', 'land', 'natur', 'havs', 'ådkgre', 'NaturThor', 'æalternativ']


Derefter kan vi bruge `corpora.Dictionary` funktionen fra `gensim` til at konvertere tokens til numeriske id'er.

In [7]:
from gensim import corpora

# Lav en dictionary - integer id per token
id2token = corpora.Dictionary(tweets_tokens) # integer id per word

print("\n")
print(id2token)



Dictionary(10688 unique tokens: ['Danmark', 'NaturThor', 'biodiversitetsmål', 'havs', 'land']...)


`id2token` er nu en `gensim.corpora.Dictionary`. Denne indeholder forskellige metoder til at konvertere tekstdata. Blandt andet indeholder den metoden `.doc2bow()`, som danner en "term-document frequecy" (optælling af ord i teksten). ("bow" er kort for "bag-of-words", da optællingen netop ikke tager højde for rækkefølgen).

Resultatet er en liste af tuples, hvor hvert tuple består af optælling af hvert token i teksten. 

Herunder bruges en "list comprehension" (et forsimplet for loop, der returnerer en liste), til at konvertere alle tweets:

In [8]:
# Danner gensim corpus - optælling af tokens per tekst
tweets_corpus = [id2token.doc2bow(tokens) for tokens in tweets_tokens] # bag-of-word(bow) tuples for hver text - (token-id, optælling)

print(tweets_corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1)]


Vi kan tjekke, hvilke ord ligger bag id'erne, med følgende kode:

In [9]:
[(id2token[id], freq) for id, freq in tweets_corpus[0]]

[('Danmark', 1),
 ('NaturThor', 1),
 ('biodiversitetsmål', 1),
 ('havs', 1),
 ('land', 1),
 ('miljøministerium', 1),
 ('natur', 2),
 ('vej', 1),
 ('ådkgre', 1),
 ('æalternativ', 1)]

Da vi ikke har ændret rækkefølgen, kan vi også finde tweetet i vores dataframe:

In [10]:
tweets_df.loc[0, 'full_text']

'Er på vej i miljøministeriet for at foreslå at vi laver biodiversitetsmål for Danmark\n\n10% strengt beskyttet natur til lands i 2030 mod ca 1% i dag\n\n20% strengt beskyttet natur til havs i 2030 mod ca 2% i dag\n\nSå har man noget at måle os på. #dkgreen @NaturThor @alternativet_ https://t.co/auA5QnEVkV'

### Opbygning af LDA model

Data er nu klar til at indgå i en gensim LDA model. Modellens parametre kan justeres på forskellig vis, men vi kører her modellem med "standardindstillingerne".

Modellen skal bruge corpus, dictionary og antal topics (hvis ikke man specificerer topics, kører den med 100 topics - se evt. `?gensim.models.ldamodel.LdaModel` Her køres med 10 topics:

In [11]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(corpus=tweets_corpus,  # Corpus af tekster
                    id2word=id2token,       # Dictionary af ord-id mapping
                     num_topics=10,          # Antal topics
                     random_state=142)       # Sætter seed - sikrer samme resultat

Når modellen er kørt, kan vi inspicere, hvilke ord, der er mest sandsynlige inden for hvert topic (vi bruger `pprint()` for at gøre printet lidt mere overskueligt):

In [12]:
from pprint import pprint

pprint(lda_model.show_topics(formatted=False)) # Viser de 10 mest sandsynlige ord per topic

[(0,
  [('barn', 0.015348122),
   ('dansker', 0.014420724),
   ('forslag', 0.0139040295),
   ('grøn', 0.010408723),
   ('æalternativ', 0.009762153),
   ('krone', 0.008521345),
   ('æpolitike', 0.007819455),
   ('Danmark', 0.0073938933),
   ('milliard', 0.006804632),
   ('dansk', 0.0066727805)]),
 (1,
  [('statsminister', 0.0141492635),
   ('dansk', 0.012120124),
   ('tid', 0.008486024),
   ('Mette', 0.00844524),
   ('blå', 0.0074493303),
   ('regering', 0.0072334935),
   ('Danmark', 0.0066043027),
   ('ådkmedie', 0.006548233),
   ('eksempel', 0.006375361),
   ('fald', 0.006153698)]),
 (2,
  [('vigtig', 0.0091714915),
   ('høj', 0.00891667),
   ('Heunicke', 0.008757982),
   ('PiaOlsen', 0.0059946035),
   ('enig', 0.0059862),
   ('København', 0.0049921917),
   ('stemme', 0.0047079097),
   ('opmærksom', 0.004450018),
   ('JanEJoergensen', 0.00441539),
   ('uge', 0.004310148)]),
 (3,
  [('Danmark', 0.01113594),
   ('regering', 0.009739464),
   ('første', 0.0076123094),
   ('natur', 0.00679

### Brug af LDA model

Med modellen defineret, kan vi anvende modellen på data igen til at fastslå, hvilke topics er mest fremtrædende i et tweet:

In [13]:
print(tweets_df.loc[0, 'full_text'])

Er på vej i miljøministeriet for at foreslå at vi laver biodiversitetsmål for Danmark

10% strengt beskyttet natur til lands i 2030 mod ca 1% i dag

20% strengt beskyttet natur til havs i 2030 mod ca 2% i dag

Så har man noget at måle os på. #dkgreen @NaturThor @alternativet_ https://t.co/auA5QnEVkV


In [14]:
lda_model[tweets_corpus[0]]

[(4, 0.8695208), (6, 0.063729264)]

Outputtet viser - groft sagt - at tweetet består af 87% topic 4 og 6,4% topic 6. Topic 4 er altså det mest dominerende topic i tweetet.

### Fortolkning af LDA modeller

LDA modeller kan hjælpe os til at finde umiddelbare temaer. Modellen kan dog ikke fortælle, hvad disse temaer egentlig indfanger. At arbejde med topic models er derfor ofte en kvali-kvantitativ proces, hvor man bruger LDA modellen til at udlede temaer, og derefter kvalitativt forsøger at udelde, hvad disse temaer indfanger. 

I denne proces finder man ofte også ud af, at visse topics indfanger det samme. På den måde er det også en iterativ proces, hvor man tilpasser modellen til bedre at indfange meningsfulde temaer i teksterne.

# ØVELSE 2: Topic models i Python (reddit data)

I skal nu selv lave en topic model for. I kan enten arbejde med egne data eller reddit kommentarerne fra r/denmark datasættet: [reddit_rdenmark-comments_01032021-08032021_long_tokenized.csv](https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/reddit_rdenmark-comments_01032021-08032021_long_tokenized.csv).

1. Tokenize data, hvis ikke det allerede er tokenized (dette blev gennemgået i lektion 5)
2. Dan et "gensim corpus" ud fra lister af tokens i data
2. Dan en topic model over tekstdata (bestem selv antal topics)
3. Inspicér de mest sandsynlige ord i hvert topic (`lda_model.show_topics(formatted=False)`)
4. Hvad kan I bruge modellen til?

**Hint:** I reddit datasættet er kommentarer allerede tokenized (kolonnen "tokens"). Tokens er gemt som lister af tokens, men for at Python kan læse dem som liste, skal I bruge `ast.literaL_eval` funktionen.

# Prævalente topics i tekst (supplerende materiale)

Vi har set på, hvordan vi kan lave en LDA model til at finde umiddelbare temaer i en tekst. Det, som kunne være interessant at se på, er, hvordan disse topics opstår i data. 

Dette kan hjælpe os til at blive klogere på, hvad disse topics egentlig indfanger, da vi på den måde kan finde tekster, hvor bestemte topics er meget prævalente.

I det følgende laves en funktion, der finder det mest prævalente topic i et tweet, sammen med dens sandsynlighed/"prævalens-score" samt nøgleordene for det topic generelt. Disse oplysinger tilføjes derefter som kolonner til datasættet med tweets.

In [19]:
# Function for getting dominant topic for one corpus entry (bag-of-word tuples - bowt)
def get_dominant_topic(text_bowt, ldamodel = lda_model):
    
    dominant_topic_dict = dict()
    
    topics_doc = ldamodel[text_bowt]
    
    dominant_topic = sorted(topics_doc, key = lambda t: t[1], reverse = True)[0]
    topic_num = dominant_topic[0]
    topic_prob = dominant_topic[1]
    
    topic_kws = [word for word, prop in ldamodel.show_topic(topic_num)]
    
    dominant_topic_dict['dominant_topic'] = topic_num
    dominant_topic_dict['topic_probability'] = topic_prob
    dominant_topic_dict['topic_keywords'] = topic_kws
    
    return(dominant_topic_dict) # Note that domninant topic info is returned as dictionary

In [20]:
# Creating list of dictionaries - one dictionary contatining dominant topic info for each corpus entry
corpus_dominant_topics = list()

for tweet_bowt in tweets_corpus:
    dominant_topic = get_dominant_topic(tweet_bowt)
    corpus_dominant_topics.append(dominant_topic)

print(corpus_dominant_topics[0])

{'dominant_topic': 4, 'topic_probability': 0.8698593, 'topic_keywords': ['dansk', 'samme', 'ådkgre', 'Danmark', 'godt', 'grøn', 'skridt', 'måde', 'fremtid', 'spørgsmål']}


In [21]:
# Add the dominant topic list as a series to the original data frame (NOTE: This assumes that order of entries has not been changed)    
tweets_df['dominant_topic_dict'] = pd.Series(corpus_dominant_topics)

In [22]:
# Spreading dominant topic dictionaries into columns using json_normalize
tweets_df_topics = pd.merge(tweets_df, pd.json_normalize(tweets_df['dominant_topic_dict']), left_index=True, right_index=True)

In [23]:
# Check resultS
tweets_df_topics.head()

,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,urls,user_followers_count,party,tokens,dominant_topic_dict,dominant_topic,topic_probability,topic_keywords
0,2020-10-21 14:48:39+00:00,1318927184111730700,Er på vej i miljøministeriet for at foreslå at...,False,13,47,False,False,False,['dkgreen'],[],4064,Alternativet,"[vej, miljøministerium, biodiversitetsmål, Dan...","{'dominant_topic': 4, 'topic_probability': 0.8...",4,0.869859,"[dansk, samme, ådkgre, Danmark, godt, grøn, sk..."
1,2019-06-02 20:03:20+00:00,1135275725592891400,@nielscallesoe @helenehagel @alternativet_ Det...,False,0,1,False,False,False,[],[],4064,Alternativet,"[æalternativ, nettofordel, klima, boring, Nord...","{'dominant_topic': 8, 'topic_probability': 0.8...",8,0.814442,"[regering, Danmark, valg, enig, lille, tid, pa..."
2,2016-03-10 09:07:52+00:00,707855478320189400,"Vi står sammen, smiler Løkke på KL-topmøde og ...",False,13,14,False,False,False,"['dkpol', 'KLtop16']",[],4064,Alternativet,"[Løkke, KL-topmøde, milliard, kommune, æaltern...","{'dominant_topic': 0, 'topic_probability': 0.8...",0,0.849815,"[barn, dansker, forslag, grøn, æalternativ, kr..."
3,2019-04-07 19:59:03+00:00,1114980930467315700,@AnnaBylov @EU_Spring @rasmusnordqvist 💚,False,0,2,False,False,False,[],[],4064,Alternativet,[eu_spring],"{'dominant_topic': 3, 'topic_probability': 0.5...",3,0.547961,"[Danmark, regering, første, natur, sundhedsvæs..."
4,2017-05-28 09:59:26+00:00,868768670427828200,Der er ikke noget alternativ til at Alternativ...,False,6,28,False,False,False,['LMÅ17'],"[{'url': 'https://t.co/3MCdZZGKRq', 'expanded_...",4064,Alternativet,"[alternativ, alternativ, klima, æalternativ]","{'dominant_topic': 9, 'topic_probability': 0.4...",9,0.459050,"[gang, barn, debat, politisk, enig, regering, ..."
